In [177]:
import libsumo

PHASES_COUNT = 8
CONFIG_PATH = "./sumo_conf/scenario1/osm.sumocfg"
MIN_TIME = 5
MAX_TIME = 60


class SumoContext:
    def __enter__(self):
        libsumo.start(["sumo", "-c", CONFIG_PATH])
        return libsumo

    def __exit__(self, exc_type, exc_val, exc_tb):
        libsumo.close()


def base_program_logic():
    with SumoContext() as sumo:
        junction_tl_id = sumo.trafficlight_getIDList()[0]
        logic = sumo.trafficlight_getAllProgramLogics(junction_tl_id)[0]
        return junction_tl_id, logic


JUNCTION_TL_ID, LOGIC = base_program_logic()

In [178]:
with SumoContext() as sumo:
    total_wt = 0
    total_veh = 0
    for i in range(3600):
        sumo.simulationStep()
        for veh_id in libsumo.vehicle.getIDList():
           if libsumo.vehicle.getSpeed(veh_id) < 0.1:
                total_wt += 1
        total_veh += libsumo.simulation.getArrivedNumber()
    phases_duration = list(map(lambda x: x.duration, sumo.trafficlight_getAllProgramLogics(sumo.trafficlight_getIDList()[0])[0].phases))
    print("Default solution: \nX = %s\nF = %s" % (phases_duration, total_wt / total_veh))

Default solution: 
X = [33.0, 3.0, 6.0, 3.0, 33.0, 3.0, 6.0, 3.0]
F = 13.309328968903436


In [179]:
def fitness(durations: list[int]):
    with SumoContext() as sumo:
        phases = LOGIC.getPhases()
        for i in range(PHASES_COUNT):
            phases[i].duration = float(durations[i])
            phases[i].minDur = float(durations[i])
            phases[i].maxDur = float(durations[i])
        sumo.trafficlight.setProgramLogic(JUNCTION_TL_ID, LOGIC)
        total_wt = 0
        total_veh = 0
        for i in range(3600):
            sumo.simulationStep()
            for veh_id in libsumo.vehicle.getIDList():
               if libsumo.vehicle.getSpeed(veh_id) < 0.1:
                    total_wt += 1
            total_veh += libsumo.simulation.getArrivedNumber()
        return total_wt / total_veh

In [182]:
from pymoo.core.problem import ElementwiseProblem
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.optimize import minimize
from pymoo.operators.sampling.rnd import IntegerRandomSampling
from pymoo.algorithms.moo.nsga2 import NSGA2
import numpy as np

class SumoProblem(ElementwiseProblem):
    def __init__(self):
        super().__init__(n_var=PHASES_COUNT, n_obj=1, xl=np.array([MIN_TIME] * PHASES_COUNT), xu=np.array([MAX_TIME] * PHASES_COUNT))

    def _evaluate(self, x, out, *args, **kwargs):
        out["F"] = fitness(x)
    
problem = SumoProblem()
algorithm = GA(pop_size=30, eliminate_duplicates=True, sampling=IntegerRandomSampling())

In [181]:
res = minimize(problem, algorithm, seed=1, verbose=True)
print("Best solution found: \nX = %s\nF = %s" % (res.X, res.F))

n_gen  |  n_eval  |     f_avg     |     f_min    
     1 |       30 |  1.225485E+02 |  5.295886E+01
     2 |       60 |  7.641458E+01 |  4.548880E+01
     3 |       90 |  5.612833E+01 |  3.258848E+01
     4 |      120 |  4.567340E+01 |  3.235815E+01
     5 |      150 |  3.973903E+01 |  3.196455E+01
     6 |      180 |  3.542030E+01 |  3.117204E+01
     7 |      210 |  3.250465E+01 |  2.923529E+01
     8 |      240 |  3.159083E+01 |  2.923529E+01
     9 |      270 |  3.082351E+01 |  2.733141E+01
    10 |      300 |  2.952491E+01 |  2.697286E+01
    11 |      330 |  2.877737E+01 |  2.694819E+01
    12 |      360 |  2.796698E+01 |  2.608717E+01
    13 |      390 |  2.680706E+01 |  2.476108E+01
    14 |      420 |  2.607729E+01 |  2.430731E+01
    15 |      450 |  2.529048E+01 |  2.173350E+01
    16 |      480 |  2.448546E+01 |  2.173350E+01
    17 |      510 |  2.359993E+01 |  2.173350E+01
    18 |      540 |  2.260669E+01 |  2.139719E+01
    19 |      570 |  2.213457E+01 |  2.082319E+01


In [184]:
nsga2 = NSGA2(
    pop_size=100,
    eliminate_duplicates=True
)

nsga2_res = minimize(problem, nsga2, seed=1, verbose=True)
print("Best solution found: \nX = %s\nF = %s" % (nsga2_res.X, nsga2_res.F))

n_gen  |  n_eval  | n_nds  |      eps      |   indicator  
     1 |      100 |      1 |             - |             -
     2 |      200 |      1 |  3.5504396974 |         ideal
     3 |      300 |      1 |  2.4386831276 |         ideal
     4 |      400 |      1 |  6.4743540862 |         ideal
     5 |      500 |      1 |  0.000000E+00 |             f
     6 |      600 |      1 |  3.1359816581 |         ideal
     7 |      700 |      1 |  0.000000E+00 |             f
     8 |      800 |      1 |  5.4163818858 |         ideal
     9 |      900 |      1 |  1.0645532248 |         ideal
    10 |     1000 |      1 |  1.8231324373 |         ideal
    11 |     1100 |      1 |  2.0103756672 |         ideal
    12 |     1200 |      1 |  0.000000E+00 |             f
    13 |     1300 |      1 |  0.2549504950 |         ideal
    14 |     1400 |      1 |  0.6852109241 |         ideal
    15 |     1500 |      1 |  0.4245250495 |         ideal
    16 |     1600 |      1 |  0.8408185901 |         ide